### Dataset

In [2]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

In [3]:
import numpy as np
import pandas as pd
df = pd.read_csv('course_lead_scoring.csv')
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,NaN,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NaN,5,71016.0,self_employed,north_america,0,0.25,1


In [5]:
# Checking missing values
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [7]:
# Checking data types
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [9]:
# Checking the number of unique values to determine whether some variables are better be put as numerical or categorical
df.nunique()

lead_source                    5
industry                       7
number_of_courses_viewed      10
annual_income               1267
employment_status              4
location                       7
interaction_count             12
lead_score                   101
converted                      2
dtype: int64

In [44]:
# Defining numerical and categorical variables
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
categorical = ['lead_source', 'industry', 'employment_status', 'location']

In [27]:
# Replacing numerical missing values with 0.0
df[numerical] = df[numerical].fillna(0.0)
df[numerical]

,number_of_courses_viewed,annual_income,interaction_count,lead_score
0,1,79450.0,4,0.94
1,1,46992.0,1,0.80
2,5,78796.0,3,0.69
3,2,83843.0,1,0.87
4,3,85012.0,3,0.62
...,...,...,...,...
1457,1,0.0,4,0.53
1458,3,65259.0,2,0.24
1459,1,45688.0,3,0.02
1460,5,71016.0,0,0.25


In [25]:
df[numerical].isna().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
dtype: int64

In [28]:
# Replacing categorical missing values with 'NA'
df[categorical] = df[categorical].fillna('NA')
df[categorical]

,lead_source,industry,employment_status,location,converted
0,paid_ads,NA,unemployed,south_america,1
1,social_media,retail,employed,south_america,0
2,events,healthcare,unemployed,australia,1
3,paid_ads,retail,NA,australia,0
4,referral,education,self_employed,europe,1
...,...,...,...,...,...
1457,referral,manufacturing,self_employed,north_america,1
1458,referral,technology,student,europe,1
1459,paid_ads,technology,student,north_america,1
1460,referral,NA,self_employed,north_america,1


In [29]:
df[categorical].isnull().sum()

lead_source          0
industry             0
employment_status    0
location             0
converted            0
dtype: int64

### Question 1. Mode for Industry

In [30]:
df.industry.mode()

0    retail
Name: industry, dtype: object

Answer 1: retail

### Question 2. Biggest correlation

In [31]:
# Calculating correlation between numerical values
df[numerical].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


Answer 2: annual_income and interaction_count (0.027036)

### Split data

In [33]:
# Split your data in train/val/test sets with 60%/20%/20% distribution.
# Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [34]:
# Reseting the index of datasets
df_val = df_val.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [35]:
# Defining what values we need to predict
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [36]:
# Make sure that the target value y is not in your dataframe.
# Deleting the value to predict to avoid using it in the model
del df_train['converted']
del df_val['converted']
del df_test['converted']

### Question 3. Mutual information score

In [38]:
# Importing a function from SciKit learn that implements MI
from sklearn.metrics import mutual_info_score

In [42]:
# Creating the function to learn which ones are more important
def mutual_info_converted_score(series):
    return mutual_info_score(series, y_train)

In [45]:
# Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
mi = df_train[categorical].apply(mutual_info_converted_score)
mi.sort_values(ascending=False)

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64

Answer 4: lead_score (0.035396)

### Question 4. Accuracy